In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.metrics import Accuracy
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import model_selection
#https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#DATA PREPARATION
raw_datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR dataset Time train.csv")
number_of_day = raw_datas.shape[0]/86400
print("Total data = " + str(raw_datas.shape[0]) + " detik")
print("Total hari dalam data = " + str(int(number_of_day)) + " hari")

Total data = 604800 detik
Total hari dalam data = 7 hari


In [ ]:
#EXTRACTING DATA

datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR dataset Duration train.csv")
file = open("/content/drive/MyDrive/Dann Dataset/PIR last 30 list dataset train.csv","w")
for z in range(30):
  file.write("Durasi")
  file.write(str(z+1))
  for y in range(4):
    file.write(",Data")
    file.write(str(z+1))
    file.write("|")
    file.write("Pir")
    file.write(str(y+1))
  file.write(",")
file.write("Gerakan")
file.write("\n")

percent1 = round(datas.shape[0]/100)
percentage = 0

for i in range(datas.shape[0]):
  if(i%percent1 == 0):
    print(str(percentage)+"%")
    percentage+=1
  if(i < 29):
    for u in range(29-i):
      file.write("0,0,0,0,0,")
    for e in range(i+1):
      for t in range(5):
        file.write(str(datas.iloc[e][t]))
        file.write(",") 
  else:
    for a in range(30):
      for o in range(5):
        file.write(str(datas.iloc[i-(29-a)][o]))
        file.write(",")
  file.write(datas.iloc[i][5])
  file.write("\n")

print("100%")
file.close()

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
100%


In [ ]:
#BEFORE EXTRACTION FEATURE
datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR dataset Duration train.csv", sep=';', header=0)
print(datas)

       Durasi,Pir_1,Pir2,Pir3,Pir4,Gerakan
0                        37,0,0,0,0,NORMAL
1                         1,0,0,1,0,NORMAL
2                         2,0,1,0,0,NORMAL
3                         2,0,0,0,1,NORMAL
4                         4,0,1,0,0,NORMAL
...                                    ...
200284                 1,0,1,0,0,LOITERING
200285                 1,1,0,0,0,LOITERING
200286                 1,0,1,0,0,LOITERING
200287                 1,1,0,0,0,LOITERING
200288                 1,1,0,1,0,LOITERING

[200289 rows x 1 columns]



- data per 30 sampel
- data yang sama 30 sampel dijadiin 1 row 

In [ ]:
#AFTER EXTRACTION FEATURE
datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR last 30 list dataset train.csv", sep=';', header=0)
print(datas)

       Durasi1,Data1|Pir1,Data1|Pir2,Data1|Pir3,Data1|Pir4,Durasi2,Data2|Pir1,Data2|Pir2,Data2|Pir3,Data2|Pir4,Durasi3,Data3|Pir1,Data3|Pir2,Data3|Pir3,Data3|Pir4,Durasi4,Data4|Pir1,Data4|Pir2,Data4|Pir3,Data4|Pir4,Durasi5,Data5|Pir1,Data5|Pir2,Data5|Pir3,Data5|Pir4,Durasi6,Data6|Pir1,Data6|Pir2,Data6|Pir3,Data6|Pir4,Durasi7,Data7|Pir1,Data7|Pir2,Data7|Pir3,Data7|Pir4,Durasi8,Data8|Pir1,Data8|Pir2,Data8|Pir3,Data8|Pir4,Durasi9,Data9|Pir1,Data9|Pir2,Data9|Pir3,Data9|Pir4,Durasi10,Data10|Pir1,Data10|Pir2,Data10|Pir3,Data10|Pir4,Durasi11,Data11|Pir1,Data11|Pir2,Data11|Pir3,Data11|Pir4,Durasi12,Data12|Pir1,Data12|Pir2,Data12|Pir3,Data12|Pir4,Durasi13,Data13|Pir1,Data13|Pir2,Data13|Pir3,Data13|Pir4,Durasi14,Data14|Pir1,Data14|Pir2,Data14|Pir3,Data14|Pir4,Durasi15,Data15|Pir1,Data15|Pir2,Data15|Pir3,Data15|Pir4,Durasi16,Data16|Pir1,Data16|Pir2,Data16|Pir3,Data16|Pir4,Durasi17,Data17|Pir1,Data17|Pir2,Data17|Pir3,Data17|Pir4,Durasi18,Data18|Pir1,Data18|Pir2,Data18|Pir3,Data18|Pir4,Durasi19,Dat

In [ ]:
# LOAD DATASET
feature_data =pd.read_csv("drive/MyDrive/Dann Dataset/PIR last 30 list dataset train.csv")

# ADD LABEL ENCODER
label_encoder = LabelEncoder()
feature_data['Gerakan'] = label_encoder.fit_transform(feature_data['Gerakan'])
datas_labels = label_encoder.classes_

In [ ]:
# LIST OF LABELS
print(datas_labels)

['LOITERING' 'NORMAL']


In [ ]:
x = feature_data.drop('Gerakan',axis=1)
y = feature_data['Gerakan']

# RESHAPE DATASET
x = x.values.reshape(-1,30,5)   #INPUT 4
y = y.values.reshape((-1,1))

# SPLIT DATASET
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) #Split Dataset Train 70% dan Test 30%

# CHECK THE SHAPE
print("Train:", x_train.shape , y_train.shape)
print("Test:",x_test.shape , y_test.shape)

# SET TOTAL FOLD
total_fold = 5

Train: (140202, 30, 5) (140202, 1)
Test: (60087, 30, 5) (60087, 1)


In [ ]:
#RNN BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4), #metode optimizer default
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6899 - accuracy: 0.5538 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.6212 - accuracy: 0.6593 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.5638 - accuracy: 0.7046 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.5219 - accuracy: 0.7327 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.4455 - accuracy: 0.7994 - 34s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3406 - accuracy: 0.8649 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2705 - accuracy: 0.8980 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2274 - accuracy: 0.9158 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.2015 - accuracy: 0.9262 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1800 - accuracy: 0.9342 - 32s/epoch - 29ms/step
accuracy: 93.97%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6630 - accuracy: 0.6039 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.5908 - accuracy: 0.6727 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 34s - loss: 0.5418 - accuracy: 0.7169 - 34s/epoch - 30ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4706 - accuracy: 0.7787 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3756 - accuracy: 0.8439 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3014 - accuracy: 0.8834 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2508 - accuracy: 0.9084 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2174 - accuracy: 0.9223 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1942 - accuracy: 0.9308 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1806 - accuracy: 0.9348 - 32s/epoch - 29ms/step
accuracy: 94.49%
Epoch 1/10


...

1122/1122 - 34s - loss: 0.6709 - accuracy: 0.5779 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 34s - loss: 0.5954 - accuracy: 0.6708 - 34s/epoch - 30ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.5254 - accuracy: 0.7317 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4293 - accuracy: 0.8070 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3463 - accuracy: 0.8575 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.2899 - accuracy: 0.8870 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2469 - accuracy: 0.9064 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2113 - accuracy: 0.9210 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1836 - accuracy: 0.9318 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 34s - loss: 0.1669 - accuracy: 0.9375 - 34s/epoch - 30ms/step
accuracy: 92.52%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6997 - accuracy: 0.5573 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.5918 - accuracy: 0.6722 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.5351 - accuracy: 0.7195 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4769 - accuracy: 0.7723 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3820 - accuracy: 0.8423 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3132 - accuracy: 0.8768 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2652 - accuracy: 0.8988 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 34s - loss: 0.2255 - accuracy: 0.9159 - 34s/epoch - 30ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1987 - accuracy: 0.9266 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1801 - accuracy: 0.9334 - 32s/epoch - 29ms/step
accuracy: 93.16%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.7001 - accuracy: 0.5429 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.6473 - accuracy: 0.6132 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.6020 - accuracy: 0.6686 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.5624 - accuracy: 0.7003 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.5179 - accuracy: 0.7380 - 34s/epoch - 31ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.4337 - accuracy: 0.8112 - 33s/epoch - 30ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.3230 - accuracy: 0.8783 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2622 - accuracy: 0.9038 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.2248 - accuracy: 0.9178 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.2006 - accuracy: 0.9267 - 32s/epoch - 29ms/step
accuracy: 91.49%
Model Accuracy List:  [93.97311210632324, 94.49021220207214, 92.52139925956726, 93.1562066078186, 91.48716330528259]
93.13% (+/- 1.06%)


In [ ]:
#RNN AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 36s - loss: 0.1879 - accuracy: 0.9173 - 36s/epoch - 32ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0607 - accuracy: 0.9769 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0375 - accuracy: 0.9857 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0326 - accuracy: 0.9870 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0579 - accuracy: 0.9814 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 35s - loss: 0.0320 - accuracy: 0.9877 - 35s/epoch - 31ms/step
Epoch 7/10
1122/1122 - 35s - loss: 0.0229 - accuracy: 0.9909 - 35s/epoch - 31ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0221 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0223 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0240 - accuracy: 0.9908 - 33s/epoch - 29ms/step
accuracy: 99.44%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.1944 - accuracy: 0.9121 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0534 - accuracy: 0.9799 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 34s - loss: 0.0375 - accuracy: 0.9854 - 34s/epoch - 30ms/step
Epoch 4/10
1122/1122 - 34s - loss: 0.0293 - accuracy: 0.9881 - 34s/epoch - 30ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0281 - accuracy: 0.9887 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0246 - accuracy: 0.9898 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0226 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0213 - accuracy: 0.9914 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0232 - accuracy: 0.9910 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0161 - accuracy: 0.9932 - 33s/epoch - 29ms/step
accuracy: 99.42%
Epoch 1/10


...

1122/1122 - 36s - loss: 0.1704 - accuracy: 0.9220 - 36s/epoch - 32ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0508 - accuracy: 0.9807 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.0350 - accuracy: 0.9865 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0395 - accuracy: 0.9854 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0243 - accuracy: 0.9904 - 33s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0257 - accuracy: 0.9896 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0266 - accuracy: 0.9896 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 34s - loss: 0.0205 - accuracy: 0.9919 - 34s/epoch - 31ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0219 - accuracy: 0.9912 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0205 - accuracy: 0.9913 - 33s/epoch - 29ms/step
accuracy: 99.10%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.1727 - accuracy: 0.9257 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0643 - accuracy: 0.9763 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0462 - accuracy: 0.9822 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0328 - accuracy: 0.9870 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.0301 - accuracy: 0.9879 - 34s/epoch - 31ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0273 - accuracy: 0.9890 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0247 - accuracy: 0.9900 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0242 - accuracy: 0.9901 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0194 - accuracy: 0.9923 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0179 - accuracy: 0.9928 - 33s/epoch - 29ms/step
accuracy: 99.24%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.2104 - accuracy: 0.9060 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 34s - loss: 0.0566 - accuracy: 0.9788 - 34s/epoch - 30ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0379 - accuracy: 0.9856 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0406 - accuracy: 0.9849 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0305 - accuracy: 0.9877 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0258 - accuracy: 0.9897 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0202 - accuracy: 0.9920 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0227 - accuracy: 0.9910 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0199 - accuracy: 0.9914 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 34s - loss: 0.0189 - accuracy: 0.9925 - 34s/epoch - 30ms/step
accuracy: 99.59%
Model Accuracy List:  [99.4365394115448, 99.41514134407043, 99.09771680831909, 99.23680424690247, 99.58987236022949]
99.36% (+/- 0.17%)


In [ ]:
#GRU BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10
1122/1122 - 158s - loss: 0.6897 - accuracy: 0.5373 - 158s/epoch - 141ms/step
Epoch 2/10
1122/1122 - 131s - loss: 0.6679 - accuracy: 0.5809 - 131s/epoch - 117ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6613 - accuracy: 0.6095 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 114s - loss: 0.6561 - accuracy: 0.6216 - 114s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 116s - loss: 0.6515 - accuracy: 0.6264 - 116s/epoch - 103ms/step
Epoch 6/10
1122/1122 - 116s - loss: 0.6474 - accuracy: 0.6294 - 116s/epoch - 103ms/step
Epoch 7/10
1122/1122 - 114s - loss: 0.6435 - accuracy: 0.6315 - 114s/epoch - 102ms/step
Epoch 8/10
1122/1122 - 117s - loss: 0.6398 - accuracy: 0.6341 - 117s/epoch - 104ms/step
Epoch 9/10
1122/1122 - 114s - loss: 0.6362 - accuracy: 0.6367 - 114s/epoch - 102ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.6326 - accuracy: 0.6394 - 116s/epoch - 104ms/step
accuracy: 63.85%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 120s - loss: 0.6881 - accuracy: 0.5133 - 120s/epoch - 107ms/step
Epoch 2/10
1122/1122 - 114s - loss: 0.6755 - accuracy: 0.5149 - 114s/epoch - 102ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6673 - accuracy: 0.5721 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 115s - loss: 0.6607 - accuracy: 0.6107 - 115s/epoch - 103ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.6551 - accuracy: 0.6215 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 114s - loss: 0.6502 - accuracy: 0.6273 - 114s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.6458 - accuracy: 0.6300 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 117s - loss: 0.6415 - accuracy: 0.6322 - 117s/epoch - 104ms/step
Epoch 9/10
1122/1122 - 114s - loss: 0.6375 - accuracy: 0.6347 - 114s/epoch - 102ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.6335 - accuracy: 0.6367 - 116s/epoch - 103ms/step
accuracy: 63.93%


Epoch 1/10


...

1122/1122 - 117s - loss: 0.6908 - accuracy: 0.5096 - 117s/epoch - 104ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.6762 - accuracy: 0.5405 - 116s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6682 - accuracy: 0.5808 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 115s - loss: 0.6615 - accuracy: 0.6097 - 115s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.6559 - accuracy: 0.6161 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 115s - loss: 0.6510 - accuracy: 0.6215 - 115s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.6465 - accuracy: 0.6247 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.6421 - accuracy: 0.6277 - 114s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 116s - loss: 0.6380 - accuracy: 0.6308 - 116s/epoch - 103ms/step
Epoch 10/10
1122/1122 - 113s - loss: 0.6341 - accuracy: 0.6331 - 113s/epoch - 101ms/step
accuracy: 63.41%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 120s - loss: 0.6846 - accuracy: 0.5129 - 120s/epoch - 107ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.6721 - accuracy: 0.5434 - 116s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 114s - loss: 0.6648 - accuracy: 0.5658 - 114s/epoch - 102ms/step
Epoch 4/10
1122/1122 - 116s - loss: 0.6591 - accuracy: 0.5809 - 116s/epoch - 104ms/step
Epoch 5/10
1122/1122 - 114s - loss: 0.6541 - accuracy: 0.5917 - 114s/epoch - 102ms/step
Epoch 6/10
1122/1122 - 116s - loss: 0.6496 - accuracy: 0.5992 - 116s/epoch - 104ms/step
Epoch 7/10
1122/1122 - 116s - loss: 0.6456 - accuracy: 0.6060 - 116s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.6417 - accuracy: 0.6120 - 114s/epoch - 101ms/step
Epoch 9/10
1122/1122 - 116s - loss: 0.6381 - accuracy: 0.6175 - 116s/epoch - 103ms/step
Epoch 10/10
1122/1122 - 114s - loss: 0.6344 - accuracy: 0.6211 - 114s/epoch - 101ms/step
accuracy: 62.77%


Model Accuracy List:  [63.84579539299011, 63.92781734466553, 63.291728496551514, 63.40941786766052, 62.767475843429565]
63.45% (+/- 0.42%)


In [ ]:
#GRU AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 119s - loss: 0.1998 - accuracy: 0.9084 - 119s/epoch - 106ms/step
Epoch 2/10
1122/1122 - 117s - loss: 0.0614 - accuracy: 0.9747 - 117s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 114s - loss: 0.0339 - accuracy: 0.9871 - 114s/epoch - 102ms/step
Epoch 4/10
1122/1122 - 117s - loss: 0.1209 - accuracy: 0.9736 - 117s/epoch - 104ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.0238 - accuracy: 0.9911 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 114s - loss: 0.0193 - accuracy: 0.9927 - 114s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.0146 - accuracy: 0.9947 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 115s - loss: 0.0141 - accuracy: 0.9946 - 115s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 117s - loss: 0.0139 - accuracy: 0.9946 - 117s/epoch - 104ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.0122 - accuracy: 0.9955 - 116s/epoch - 103ms/step
accuracy: 99.58%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

1122/1122 - 118s - loss: 0.2009 - accuracy: 0.9087 - 118s/epoch - 105ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.0553 - accuracy: 0.9775 - 116s/epoch - 103ms/step
Epoch 3/10
1122/1122 - 116s - loss: 0.0317 - accuracy: 0.9880 - 116s/epoch - 103ms/step
Epoch 4/10
1122/1122 - 114s - loss: 0.0223 - accuracy: 0.9914 - 114s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 116s - loss: 0.6613 - accuracy: 0.8001 - 116s/epoch - 103ms/step
Epoch 6/10
1122/1122 - 113s - loss: 0.1165 - accuracy: 0.9584 - 113s/epoch - 101ms/step
Epoch 7/10
1122/1122 - 115s - loss: 0.0547 - accuracy: 0.9802 - 115s/epoch - 103ms/step
Epoch 8/10
1122/1122 - 115s - loss: 0.0394 - accuracy: 0.9854 - 115s/epoch - 103ms/step
Epoch 9/10
1122/1122 - 113s - loss: 0.0300 - accuracy: 0.9887 - 113s/epoch - 101ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.0250 - accuracy: 0.9903 - 116s/epoch - 103ms/step
accuracy: 98.98%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 119s - loss: 0.2170 - accuracy: 0.8975 - 119s/epoch - 106ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.0553 - accuracy: 0.9775 - 116s/epoch - 103ms/step
Epoch 3/10
1122/1122 - 113s - loss: 0.0344 - accuracy: 0.9871 - 113s/epoch - 100ms/step
Epoch 4/10
1122/1122 - 116s - loss: 0.0234 - accuracy: 0.9906 - 116s/epoch - 103ms/step
Epoch 5/10
1122/1122 - 115s - loss: 0.0185 - accuracy: 0.9926 - 115s/epoch - 102ms/step
Epoch 6/10
1122/1122 - 113s - loss: 0.0252 - accuracy: 0.9918 - 113s/epoch - 101ms/step
Epoch 7/10
1122/1122 - 115s - loss: 0.0152 - accuracy: 0.9938 - 115s/epoch - 102ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.0141 - accuracy: 0.9944 - 114s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 113s - loss: 0.0137 - accuracy: 0.9945 - 113s/epoch - 101ms/step
Epoch 10/10
1122/1122 - 115s - loss: 0.0119 - accuracy: 0.9953 - 115s/epoch - 102ms/step
accuracy: 99.33%


Model Accuracy List:  [99.5756208896637, 99.33668375015259, 98.76961708068848, 98.98002743721008, 99.33309555053711]
99.20% (+/- 0.29%)


In [ ]:
#LSTM BEFORE TUNING 

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 126s - loss: 0.6782 - accuracy: 0.5242 - 126s/epoch - 112ms/step
Epoch 2/10
1122/1122 - 124s - loss: 0.6670 - accuracy: 0.6303 - 124s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 124s - loss: 0.6571 - accuracy: 0.6869 - 124s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.6471 - accuracy: 0.6965 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.6366 - accuracy: 0.6991 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 122s - loss: 0.6260 - accuracy: 0.7011 - 122s/epoch - 109ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6154 - accuracy: 0.7016 - 125s/epoch - 112ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.6049 - accuracy: 0.7014 - 125s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.5950 - accuracy: 0.7005 - 125s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 124s - loss: 0.5862 - accuracy: 0.6995 - 124s/epoch - 111ms/step
accuracy: 70.01%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

1122/1122 - 127s - loss: 0.6917 - accuracy: 0.5785 - 127s/epoch - 113ms/step
Epoch 2/10
1122/1122 - 124s - loss: 0.6768 - accuracy: 0.6708 - 124s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.6669 - accuracy: 0.6979 - 125s/epoch - 112ms/step
Epoch 4/10
1122/1122 - 123s - loss: 0.6571 - accuracy: 0.7016 - 123s/epoch - 110ms/step
Epoch 5/10
1122/1122 - 126s - loss: 0.6469 - accuracy: 0.7022 - 126s/epoch - 112ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.6360 - accuracy: 0.7031 - 125s/epoch - 112ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6248 - accuracy: 0.7016 - 125s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.6135 - accuracy: 0.7012 - 125s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 124s - loss: 0.6026 - accuracy: 0.6996 - 124s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.5927 - accuracy: 0.6977 - 125s/epoch - 111ms/step
accuracy: 69.58%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 127s - loss: 0.6843 - accuracy: 0.5172 - 127s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.6688 - accuracy: 0.6362 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.6575 - accuracy: 0.6814 - 125s/epoch - 112ms/step
Epoch 4/10
1122/1122 - 126s - loss: 0.6467 - accuracy: 0.6913 - 126s/epoch - 112ms/step
Epoch 5/10
1122/1122 - 123s - loss: 0.6359 - accuracy: 0.6960 - 123s/epoch - 110ms/step
Epoch 6/10
1122/1122 - 124s - loss: 0.6246 - accuracy: 0.6989 - 124s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6127 - accuracy: 0.7009 - 125s/epoch - 112ms/step
Epoch 8/10
1122/1122 - 126s - loss: 0.6017 - accuracy: 0.7015 - 126s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 126s - loss: 0.5919 - accuracy: 0.7019 - 126s/epoch - 112ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.5834 - accuracy: 0.7010 - 125s/epoch - 111ms/step
accuracy: 70.45%


Model Accuracy List:  [70.0082004070282, 69.80849504470825, 69.86448168754578, 69.57560777664185, 70.44935822486877]
69.94% (+/- 0.29%)


In [ ]:
#LSTM AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 127s - loss: 0.2203 - accuracy: 0.9027 - 127s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.0972 - accuracy: 0.9645 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.0531 - accuracy: 0.9795 - 125s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.0409 - accuracy: 0.9837 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.0298 - accuracy: 0.9881 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.0297 - accuracy: 0.9890 - 125s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 124s - loss: 0.0232 - accuracy: 0.9906 - 124s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.0198 - accuracy: 0.9915 - 125s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.0179 - accuracy: 0.9928 - 125s/epoch - 112ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.0178 - accuracy: 0.9927 - 125s/epoch - 111ms/step
accuracy: 98.38%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

1122/1122 - 128s - loss: 0.2332 - accuracy: 0.8966 - 128s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.1086 - accuracy: 0.9583 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 124s - loss: 0.0581 - accuracy: 0.9771 - 124s/epoch - 110ms/step
Epoch 4/10
1122/1122 - 124s - loss: 0.0612 - accuracy: 0.9797 - 124s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.0337 - accuracy: 0.9872 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.0280 - accuracy: 0.9890 - 125s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 124s - loss: 0.0249 - accuracy: 0.9902 - 124s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 124s - loss: 0.0233 - accuracy: 0.9905 - 124s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.0243 - accuracy: 0.9902 - 125s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 124s - loss: 0.0192 - accuracy: 0.9922 - 124s/epoch - 111ms/step
accuracy: 99.44%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 129s - loss: 0.2265 - accuracy: 0.9012 - 129s/epoch - 115ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.0805 - accuracy: 0.9678 - 125s/epoch - 112ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.0721 - accuracy: 0.9748 - 125s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.0442 - accuracy: 0.9833 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 127s - loss: 0.0347 - accuracy: 0.9868 - 127s/epoch - 113ms/step
Epoch 6/10
1122/1122 - 127s - loss: 0.0276 - accuracy: 0.9893 - 127s/epoch - 113ms/step
Epoch 7/10
1122/1122 - 123s - loss: 0.0212 - accuracy: 0.9913 - 123s/epoch - 110ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.0243 - accuracy: 0.9905 - 125s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 126s - loss: 0.0160 - accuracy: 0.9936 - 126s/epoch - 113ms/step
Epoch 10/10
1122/1122 - 126s - loss: 0.0187 - accuracy: 0.9928 - 126s/epoch - 112ms/step
accuracy: 99.08%


Model Accuracy List:  [98.38094115257263, 98.70190024375916, 98.84807467460632, 99.4436502456665, 99.08345341682434]
98.89% (+/- 0.36%)


Get list of accuracy, precision, recall and f1
Pengujian Data Testing
- Load Model untuk klasifikasi data Testing
- Model sudah di Load lalu melakukan Predict terhadap data Testing
- Simpan data Hasil Predict untuk codingan selanjutnya

In [ ]:
#RNN K-FOLD BEFORE TUNING 
file_rnn_fold_before = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_before = [(pickle.load(open(file_rnn_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_before = [(loaded_model_rnn_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_before = [(np.argmax(y_pred_rnn_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_before[ypvc] = np.reshape(y_pred_values_rnn_before[ypvc],(-1,1))
result_rnn_before = [(classification_report(y_test, y_pred_values_rnn_before[res], output_dict = True)) for res in range(total_fold)]

#RNN K-FOLD AFTER TUNING 
file_rnn_fold_after = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_after = [(pickle.load(open(file_rnn_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_after = [(loaded_model_rnn_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_after = [(np.argmax(y_pred_rnn_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_after[ypvc] = np.reshape(y_pred_values_rnn_after[ypvc],(-1,1))
result_rnn_after = [(classification_report(y_test, y_pred_values_rnn_after[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD BEFORE TUNING
file_gru_fold_before = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_before = [(pickle.load(open(file_gru_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_before = [(loaded_model_gru_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_before = [(np.argmax(y_pred_gru_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_before[ypvc] = np.reshape(y_pred_values_gru_before[ypvc],(-1,1))
result_gru_before = [(classification_report(y_test, y_pred_values_gru_before[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD AFTER TUNING
file_gru_fold_after = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_after = [(pickle.load(open(file_gru_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_after = [(loaded_model_gru_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_after = [(np.argmax(y_pred_gru_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_after[ypvc] = np.reshape(y_pred_values_gru_after[ypvc],(-1,1))
result_gru_after = [(classification_report(y_test, y_pred_values_gru_after[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD BEFORE TUNING
file_lstm_fold_before = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_before = [(pickle.load(open(file_lstm_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_before = [(loaded_model_lstm_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_before = [(np.argmax(y_pred_lstm_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_before[ypvc] = np.reshape(y_pred_values_lstm_before[ypvc],(-1,1))
result_lstm_before = [(classification_report(y_test, y_pred_values_lstm_before[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD AFTER TUNING
file_lstm_fold_after = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_after = [(pickle.load(open(file_lstm_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_after = [(loaded_model_lstm_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_after = [(np.argmax(y_pred_lstm_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_after[ypvc] = np.reshape(y_pred_values_lstm_after[ypvc],(-1,1))
result_lstm_after = [(classification_report(y_test, y_pred_values_lstm_after[res], output_dict = True)) for res in range(total_fold)]

1878/1878 [==============================] - 32s 17ms/step


Buat List untuk menampung dari Hasil dari Fold untuk Precision,Recall,Accuracy

In [ ]:
precision_list_before = {}
precision_list_after = {}
recall_list_before = {}
recall_list_after = {}
accuracy_list_before = {}
accuracy_list_after = {}
f1_list_before = {}
f1_list_after = {}

Rata-rata dari Precision,Recall,F1-Score,Accuracy

Mengambil data hasil predict pada codingan sebelumnya


In [ ]:
for sco in range(total_fold):
  precision_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['precision'] , result_gru_before[sco]['weighted avg']['precision'], result_lstm_before[sco]['weighted avg']['precision']]
  precision_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['precision'] , result_gru_after[sco]['weighted avg']['precision'], result_lstm_after[sco]['weighted avg']['precision']]
  recall_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['recall'] , result_gru_before[sco]['weighted avg']['recall'], result_lstm_before[sco]['weighted avg']['recall']]
  recall_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['recall'] , result_gru_after[sco]['weighted avg']['recall'], result_lstm_after[sco]['weighted avg']['recall']]
  f1_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['f1-score'] , result_gru_before[sco]['weighted avg']['f1-score'], result_lstm_before[sco]['weighted avg']['f1-score']]
  f1_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['f1-score'] , result_gru_after[sco]['weighted avg']['f1-score'], result_lstm_after[sco]['weighted avg']['f1-score']]
  accuracy_list_before[(sco+1)] = [result_rnn_before[sco]['accuracy'] , result_gru_before[sco]['accuracy'], result_lstm_before[sco]['accuracy']]
  accuracy_list_after[(sco+1)] = [result_rnn_after[sco]['accuracy'] , result_gru_after[sco]['accuracy'], result_lstm_after[sco]['accuracy']]

Menampilkan Hasil Predict Tiap Fold

In [ ]:
print("{:<5} {:<0}".format("","Precision K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Recall K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Accuracy K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","F1-Score K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

      Precision K-Fold Sebelum Tuning
Fold     RNN             GRU        LSTM      
1        0.943           0.695      0.734     
2        0.944           0.728      0.747     
3        0.929           0.718      0.745     
4        0.938           0.710      0.731     
5        0.924           0.679      0.744     


      Precision K-Fold Sesudah Tuning
Fold     RNN             GRU        LSTM      
1        0.995           0.995      0.986     
2        0.995           0.994      0.988     
3        0.991           0.989      0.988     
4        0.993           0.989      0.995     
5        0.997           0.993      0.990     


      Recall K-Fold Sebelum Tuning
Fold     RNN             GRU        LSTM      
1        0.941           0.639      0.697     
2        0.943           0.636      0.699     
3        0.925           0.633      0.700     
4        0.932           0.635      0.696     
5        0.915           0.624      0.701     


      Recall K-Fold Sesudah Tuning
Fo

DATA TESTING BEFORE & AFTER TUNING 30%

In [ ]:
#RNN 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_fold_before_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_before.sav"
pickle.dump(model, open(filename, 'wb'))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 15s - loss: 0.6898 - accuracy: 0.5532 - 15s/epoch - 31ms/step
Epoch 2/10
481/481 - 14s - loss: 0.6451 - accuracy: 0.6154 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.6134 - accuracy: 0.6538 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.5861 - accuracy: 0.6809 - 14s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.5612 - accuracy: 0.7030 - 14s/epoch - 29ms/step
Epoch 6/10
481/481 - 14s - loss: 0.5313 - accuracy: 0.7322 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.4851 - accuracy: 0.7762 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.4194 - accuracy: 0.8268 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.3614 - accuracy: 0.8595 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.3256 - accuracy: 0.8733 - 14s/epoch - 29ms/step
accuracy: 89.95%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 15s - loss: 0.7050 - accuracy: 0.5402 - 15s/epoch - 32ms/step
Epoch 2/10
481/481 - 14s - loss: 0.6649 - accuracy: 0.5901 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.6378 - accuracy: 0.6349 - 14s/epoch - 30ms/step
Epoch 4/10
481/481 - 15s - loss: 0.6126 - accuracy: 0.6676 - 15s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.5860 - accuracy: 0.6898 - 14s/epoch - 30ms/step
Epoch 6/10
481/481 - 14s - loss: 0.5598 - accuracy: 0.7108 - 14s/epoch - 30ms/step
Epoch 7/10
481/481 - 14s - loss: 0.5321 - accuracy: 0.7345 - 14s/epoch - 30ms/step
Epoch 8/10
481/481 - 14s - loss: 0.4971 - accuracy: 0.7653 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.4491 - accuracy: 0.8047 - 14s/epoch - 29ms/step
Epoch 10/10
481/481 - 14s - loss: 0.3891 - accuracy: 0.8430 - 14s/epoch - 29ms/step
accuracy: 86.49%
Epoch 1/10


...

481/481 - 16s - loss: 0.6872 - accuracy: 0.5562 - 16s/epoch - 32ms/step
Epoch 2/10
481/481 - 14s - loss: 0.6490 - accuracy: 0.6125 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.6181 - accuracy: 0.6570 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.5908 - accuracy: 0.6845 - 14s/epoch - 29ms/step
Epoch 5/10
481/481 - 14s - loss: 0.5685 - accuracy: 0.7004 - 14s/epoch - 30ms/step
Epoch 6/10
481/481 - 14s - loss: 0.5494 - accuracy: 0.7157 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.5290 - accuracy: 0.7326 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.5035 - accuracy: 0.7552 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.4670 - accuracy: 0.7836 - 14s/epoch - 29ms/step
Epoch 10/10
481/481 - 14s - loss: 0.4174 - accuracy: 0.8189 - 14s/epoch - 30ms/step
accuracy: 84.52%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 15s - loss: 0.6799 - accuracy: 0.5653 - 15s/epoch - 32ms/step
Epoch 2/10
481/481 - 14s - loss: 0.6468 - accuracy: 0.6126 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.6198 - accuracy: 0.6486 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.5900 - accuracy: 0.6804 - 14s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.5586 - accuracy: 0.7053 - 14s/epoch - 30ms/step
Epoch 6/10
481/481 - 14s - loss: 0.5299 - accuracy: 0.7263 - 14s/epoch - 30ms/step
Epoch 7/10
481/481 - 14s - loss: 0.4994 - accuracy: 0.7544 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.4629 - accuracy: 0.7871 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.4143 - accuracy: 0.8263 - 14s/epoch - 29ms/step
Epoch 10/10
481/481 - 14s - loss: 0.3693 - accuracy: 0.8521 - 14s/epoch - 29ms/step
accuracy: 84.83%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 15s - loss: 0.7431 - accuracy: 0.4914 - 15s/epoch - 32ms/step
Epoch 2/10
481/481 - 14s - loss: 0.6996 - accuracy: 0.5307 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.6716 - accuracy: 0.5703 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.6471 - accuracy: 0.6042 - 14s/epoch - 29ms/step
Epoch 5/10
481/481 - 14s - loss: 0.6243 - accuracy: 0.6335 - 14s/epoch - 30ms/step
Epoch 6/10
481/481 - 14s - loss: 0.6028 - accuracy: 0.6573 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.5826 - accuracy: 0.6781 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.5648 - accuracy: 0.6932 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.5488 - accuracy: 0.7063 - 14s/epoch - 29ms/step
Epoch 10/10
481/481 - 14s - loss: 0.5324 - accuracy: 0.7196 - 14s/epoch - 29ms/step
accuracy: 72.89%
Model Accuracy List:  [89.94840979576111, 86.48693561553955, 84.52192544937134, 84.82982516288757, 72.88841009140015]
83.74% (+/- 5.76%)


In [ ]:
#RNN 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_fold_after_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_after.sav"
pickle.dump(model, open(filename, 'wb'))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 17s - loss: 0.4240 - accuracy: 0.7843 - 17s/epoch - 35ms/step
Epoch 2/10
481/481 - 14s - loss: 0.1158 - accuracy: 0.9563 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.0732 - accuracy: 0.9716 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.0491 - accuracy: 0.9816 - 14s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.0437 - accuracy: 0.9839 - 14s/epoch - 29ms/step
Epoch 6/10
481/481 - 14s - loss: 0.0311 - accuracy: 0.9878 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.0317 - accuracy: 0.9880 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.0551 - accuracy: 0.9810 - 14s/epoch - 30ms/step
Epoch 9/10
481/481 - 14s - loss: 0.0279 - accuracy: 0.9889 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.0222 - accuracy: 0.9913 - 14s/epoch - 30ms/step
accuracy: 96.95%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 16s - loss: 0.2782 - accuracy: 0.8691 - 16s/epoch - 33ms/step
Epoch 2/10
481/481 - 14s - loss: 0.0992 - accuracy: 0.9626 - 14s/epoch - 30ms/step
Epoch 3/10
481/481 - 14s - loss: 0.0706 - accuracy: 0.9742 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.0629 - accuracy: 0.9777 - 14s/epoch - 29ms/step
Epoch 5/10
481/481 - 14s - loss: 0.0467 - accuracy: 0.9822 - 14s/epoch - 29ms/step
Epoch 6/10
481/481 - 14s - loss: 0.0413 - accuracy: 0.9843 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.0336 - accuracy: 0.9870 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.0305 - accuracy: 0.9886 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.0583 - accuracy: 0.9805 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.0278 - accuracy: 0.9891 - 14s/epoch - 30ms/step
accuracy: 97.33%
Epoch 1/10


...

481/481 - 16s - loss: 0.3092 - accuracy: 0.8540 - 16s/epoch - 33ms/step
Epoch 2/10
481/481 - 14s - loss: 0.0883 - accuracy: 0.9662 - 14s/epoch - 30ms/step
Epoch 3/10
481/481 - 14s - loss: 0.0554 - accuracy: 0.9799 - 14s/epoch - 30ms/step
Epoch 4/10
481/481 - 14s - loss: 0.0410 - accuracy: 0.9847 - 14s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.0374 - accuracy: 0.9860 - 14s/epoch - 30ms/step
Epoch 6/10
481/481 - 14s - loss: 0.0332 - accuracy: 0.9875 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.0261 - accuracy: 0.9893 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.0247 - accuracy: 0.9905 - 14s/epoch - 30ms/step
Epoch 9/10
481/481 - 14s - loss: 0.0274 - accuracy: 0.9893 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.0210 - accuracy: 0.9917 - 14s/epoch - 29ms/step
accuracy: 98.87%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 16s - loss: 0.2851 - accuracy: 0.8666 - 16s/epoch - 33ms/step
Epoch 2/10
481/481 - 14s - loss: 0.0986 - accuracy: 0.9610 - 14s/epoch - 30ms/step
Epoch 3/10
481/481 - 14s - loss: 0.0745 - accuracy: 0.9722 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.0557 - accuracy: 0.9791 - 14s/epoch - 29ms/step
Epoch 5/10
481/481 - 14s - loss: 0.0390 - accuracy: 0.9852 - 14s/epoch - 29ms/step
Epoch 6/10
481/481 - 14s - loss: 0.0366 - accuracy: 0.9863 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.0385 - accuracy: 0.9858 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.0274 - accuracy: 0.9895 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.0269 - accuracy: 0.9890 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.0251 - accuracy: 0.9901 - 14s/epoch - 29ms/step
accuracy: 98.97%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 16s - loss: 0.3088 - accuracy: 0.8509 - 16s/epoch - 32ms/step
Epoch 2/10
481/481 - 14s - loss: 0.1047 - accuracy: 0.9592 - 14s/epoch - 29ms/step
Epoch 3/10
481/481 - 14s - loss: 0.0664 - accuracy: 0.9753 - 14s/epoch - 29ms/step
Epoch 4/10
481/481 - 14s - loss: 0.0561 - accuracy: 0.9794 - 14s/epoch - 30ms/step
Epoch 5/10
481/481 - 14s - loss: 0.0417 - accuracy: 0.9843 - 14s/epoch - 29ms/step
Epoch 6/10
481/481 - 14s - loss: 0.0388 - accuracy: 0.9851 - 14s/epoch - 29ms/step
Epoch 7/10
481/481 - 14s - loss: 0.0353 - accuracy: 0.9856 - 14s/epoch - 29ms/step
Epoch 8/10
481/481 - 14s - loss: 0.0305 - accuracy: 0.9879 - 14s/epoch - 29ms/step
Epoch 9/10
481/481 - 14s - loss: 0.0246 - accuracy: 0.9907 - 14s/epoch - 30ms/step
Epoch 10/10
481/481 - 14s - loss: 0.0424 - accuracy: 0.9845 - 14s/epoch - 29ms/step
accuracy: 99.03%
Model Accuracy List:  [96.95456624031067, 97.32900857925415, 98.86826872825623, 98.96813035011292, 99.03470277786255]
98.23% (+/- 0.90%)


In [ ]:
#GRU 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_fold_before_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_before.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 55s - loss: 0.6835 - accuracy: 0.5629 - 55s/epoch - 115ms/step
Epoch 2/10
481/481 - 49s - loss: 0.6779 - accuracy: 0.5384 - 49s/epoch - 102ms/step
Epoch 3/10
481/481 - 49s - loss: 0.6736 - accuracy: 0.5355 - 49s/epoch - 102ms/step
Epoch 4/10
481/481 - 49s - loss: 0.6707 - accuracy: 0.5468 - 49s/epoch - 102ms/step
Epoch 5/10
481/481 - 49s - loss: 0.6682 - accuracy: 0.5507 - 49s/epoch - 102ms/step
Epoch 6/10
481/481 - 49s - loss: 0.6659 - accuracy: 0.5562 - 49s/epoch - 103ms/step
Epoch 7/10
481/481 - 49s - loss: 0.6638 - accuracy: 0.5631 - 49s/epoch - 102ms/step
Epoch 8/10
481/481 - 49s - loss: 0.6617 - accuracy: 0.5761 - 49s/epoch - 103ms/step
Epoch 9/10
481/481 - 49s - loss: 0.6598 - accuracy: 0.5768 - 49s/epoch - 102ms/step
Epoch 10/10
481/481 - 49s - loss: 0.6580 - accuracy: 0.5953 - 49s/epoch - 102ms/step
accuracy: 59.28%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

481/481 - 52s - loss: 0.6950 - accuracy: 0.4825 - 52s/epoch - 108ms/step
Epoch 2/10
481/481 - 49s - loss: 0.6853 - accuracy: 0.5092 - 49s/epoch - 102ms/step
Epoch 3/10
481/481 - 49s - loss: 0.6812 - accuracy: 0.5108 - 49s/epoch - 101ms/step
Epoch 4/10
481/481 - 49s - loss: 0.6778 - accuracy: 0.5107 - 49s/epoch - 101ms/step
Epoch 5/10
481/481 - 49s - loss: 0.6748 - accuracy: 0.5126 - 49s/epoch - 101ms/step
Epoch 6/10
481/481 - 49s - loss: 0.6719 - accuracy: 0.5185 - 49s/epoch - 101ms/step
Epoch 7/10
481/481 - 49s - loss: 0.6685 - accuracy: 0.5321 - 49s/epoch - 101ms/step
Epoch 8/10
481/481 - 49s - loss: 0.6653 - accuracy: 0.5548 - 49s/epoch - 101ms/step
Epoch 9/10
481/481 - 49s - loss: 0.6626 - accuracy: 0.5816 - 49s/epoch - 101ms/step
Epoch 10/10
481/481 - 49s - loss: 0.6602 - accuracy: 0.5858 - 49s/epoch - 101ms/step
accuracy: 59.18%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 52s - loss: 0.6733 - accuracy: 0.5425 - 52s/epoch - 108ms/step
Epoch 2/10
481/481 - 49s - loss: 0.6696 - accuracy: 0.5711 - 49s/epoch - 101ms/step
Epoch 3/10
481/481 - 49s - loss: 0.6665 - accuracy: 0.5869 - 49s/epoch - 101ms/step
Epoch 4/10
481/481 - 49s - loss: 0.6639 - accuracy: 0.5963 - 49s/epoch - 101ms/step
Epoch 5/10
481/481 - 49s - loss: 0.6615 - accuracy: 0.6000 - 49s/epoch - 101ms/step
Epoch 6/10
481/481 - 49s - loss: 0.6591 - accuracy: 0.6051 - 49s/epoch - 101ms/step
Epoch 7/10
481/481 - 49s - loss: 0.6569 - accuracy: 0.6092 - 49s/epoch - 101ms/step
Epoch 8/10
481/481 - 49s - loss: 0.6548 - accuracy: 0.6118 - 49s/epoch - 101ms/step
Epoch 9/10
481/481 - 49s - loss: 0.6528 - accuracy: 0.6124 - 49s/epoch - 101ms/step
Epoch 10/10
481/481 - 49s - loss: 0.6510 - accuracy: 0.6146 - 49s/epoch - 101ms/step
accuracy: 61.01%


Model Accuracy List:  [59.27774906158447, 58.96155834197998, 60.60580611228943, 59.182822704315186, 61.00524067878723]
59.81% (+/- 0.83%)


In [ ]:
#GRU 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_fold_after_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_after.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 53s - loss: 0.3392 - accuracy: 0.8323 - 53s/epoch - 110ms/step
Epoch 2/10
481/481 - 49s - loss: 0.1004 - accuracy: 0.9607 - 49s/epoch - 101ms/step
Epoch 3/10
481/481 - 49s - loss: 0.0736 - accuracy: 0.9679 - 49s/epoch - 101ms/step
Epoch 4/10
481/481 - 49s - loss: 0.0512 - accuracy: 0.9797 - 49s/epoch - 101ms/step
Epoch 5/10
481/481 - 49s - loss: 0.0412 - accuracy: 0.9845 - 49s/epoch - 102ms/step
Epoch 6/10
481/481 - 49s - loss: 0.0307 - accuracy: 0.9882 - 49s/epoch - 101ms/step
Epoch 7/10
481/481 - 49s - loss: 0.0254 - accuracy: 0.9901 - 49s/epoch - 102ms/step
Epoch 8/10
481/481 - 49s - loss: 0.0267 - accuracy: 0.9896 - 49s/epoch - 102ms/step
Epoch 9/10
481/481 - 49s - loss: 0.0215 - accuracy: 0.9912 - 49s/epoch - 101ms/step
Epoch 10/10
481/481 - 49s - loss: 0.0213 - accuracy: 0.9915 - 49s/epoch - 103ms/step
accuracy: 99.20%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

481/481 - 53s - loss: 0.3090 - accuracy: 0.8536 - 53s/epoch - 111ms/step
Epoch 2/10
481/481 - 49s - loss: 0.1067 - accuracy: 0.9573 - 49s/epoch - 103ms/step
Epoch 3/10
481/481 - 49s - loss: 0.0722 - accuracy: 0.9679 - 49s/epoch - 102ms/step
Epoch 4/10
481/481 - 49s - loss: 0.0558 - accuracy: 0.9779 - 49s/epoch - 101ms/step
Epoch 5/10
481/481 - 49s - loss: 0.0458 - accuracy: 0.9823 - 49s/epoch - 101ms/step
Epoch 6/10
481/481 - 49s - loss: 0.0384 - accuracy: 0.9848 - 49s/epoch - 101ms/step
Epoch 7/10
481/481 - 49s - loss: 0.0277 - accuracy: 0.9896 - 49s/epoch - 101ms/step
Epoch 8/10
481/481 - 49s - loss: 0.0263 - accuracy: 0.9892 - 49s/epoch - 101ms/step
Epoch 9/10
481/481 - 49s - loss: 0.0216 - accuracy: 0.9914 - 49s/epoch - 102ms/step
Epoch 10/10
481/481 - 49s - loss: 0.0188 - accuracy: 0.9924 - 49s/epoch - 101ms/step
accuracy: 98.29%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 53s - loss: 0.3504 - accuracy: 0.8306 - 53s/epoch - 111ms/step
Epoch 2/10
481/481 - 50s - loss: 0.1082 - accuracy: 0.9580 - 50s/epoch - 104ms/step
Epoch 3/10
481/481 - 49s - loss: 0.0767 - accuracy: 0.9665 - 49s/epoch - 102ms/step
Epoch 4/10
481/481 - 49s - loss: 0.0519 - accuracy: 0.9785 - 49s/epoch - 102ms/step
Epoch 5/10
481/481 - 49s - loss: 0.0391 - accuracy: 0.9851 - 49s/epoch - 102ms/step
Epoch 6/10
481/481 - 49s - loss: 0.0341 - accuracy: 0.9863 - 49s/epoch - 101ms/step
Epoch 7/10
481/481 - 49s - loss: 0.0290 - accuracy: 0.9889 - 49s/epoch - 101ms/step
Epoch 8/10
481/481 - 49s - loss: 0.0250 - accuracy: 0.9907 - 49s/epoch - 102ms/step
Epoch 9/10
481/481 - 49s - loss: 0.0226 - accuracy: 0.9909 - 49s/epoch - 102ms/step
Epoch 10/10
481/481 - 51s - loss: 0.0206 - accuracy: 0.9931 - 51s/epoch - 105ms/step
accuracy: 99.18%


Model Accuracy List:  [99.20119643211365, 98.56881499290466, 98.87658953666687, 98.29408526420593, 99.17616844177246]
98.82% (+/- 0.35%)


In [ ]:
#LSTM 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_fold_before_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_before.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 56s - loss: 0.6919 - accuracy: 0.5119 - 56s/epoch - 116ms/step
Epoch 2/10
481/481 - 53s - loss: 0.6842 - accuracy: 0.5163 - 53s/epoch - 109ms/step
Epoch 3/10
481/481 - 53s - loss: 0.6777 - accuracy: 0.5502 - 53s/epoch - 110ms/step
Epoch 4/10
481/481 - 56s - loss: 0.6716 - accuracy: 0.5928 - 56s/epoch - 117ms/step
Epoch 5/10
481/481 - 54s - loss: 0.6656 - accuracy: 0.6242 - 54s/epoch - 112ms/step
Epoch 6/10
481/481 - 56s - loss: 0.6595 - accuracy: 0.6517 - 56s/epoch - 116ms/step
Epoch 7/10
481/481 - 72s - loss: 0.6533 - accuracy: 0.6656 - 72s/epoch - 151ms/step
Epoch 8/10
481/481 - 74s - loss: 0.6477 - accuracy: 0.6782 - 74s/epoch - 153ms/step
Epoch 9/10
481/481 - 58s - loss: 0.6423 - accuracy: 0.6830 - 58s/epoch - 120ms/step
Epoch 10/10
481/481 - 53s - loss: 0.6369 - accuracy: 0.6868 - 53s/epoch - 110ms/step
accuracy: 69.34%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

481/481 - 56s - loss: 0.6917 - accuracy: 0.4997 - 56s/epoch - 116ms/step
Epoch 2/10
481/481 - 53s - loss: 0.6870 - accuracy: 0.5115 - 53s/epoch - 109ms/step
Epoch 3/10
481/481 - 53s - loss: 0.6820 - accuracy: 0.5197 - 53s/epoch - 109ms/step
Epoch 4/10
481/481 - 53s - loss: 0.6784 - accuracy: 0.5338 - 53s/epoch - 110ms/step
Epoch 5/10
481/481 - 53s - loss: 0.6750 - accuracy: 0.5495 - 53s/epoch - 110ms/step
Epoch 6/10
481/481 - 53s - loss: 0.6716 - accuracy: 0.5736 - 53s/epoch - 110ms/step
Epoch 7/10
481/481 - 55s - loss: 0.6682 - accuracy: 0.5955 - 55s/epoch - 114ms/step
Epoch 8/10
481/481 - 53s - loss: 0.6647 - accuracy: 0.6208 - 53s/epoch - 110ms/step
Epoch 9/10
481/481 - 54s - loss: 0.6612 - accuracy: 0.6342 - 54s/epoch - 112ms/step
Epoch 10/10
481/481 - 53s - loss: 0.6578 - accuracy: 0.6511 - 53s/epoch - 109ms/step
accuracy: 64.81%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


481/481 - 56s - loss: 0.7016 - accuracy: 0.4895 - 56s/epoch - 117ms/step
Epoch 2/10
481/481 - 53s - loss: 0.6881 - accuracy: 0.5617 - 53s/epoch - 109ms/step
Epoch 3/10
481/481 - 52s - loss: 0.6830 - accuracy: 0.6586 - 52s/epoch - 109ms/step
Epoch 4/10
481/481 - 53s - loss: 0.6795 - accuracy: 0.5811 - 53s/epoch - 109ms/step
Epoch 5/10
481/481 - 53s - loss: 0.6763 - accuracy: 0.5735 - 53s/epoch - 109ms/step
Epoch 6/10
481/481 - 53s - loss: 0.6733 - accuracy: 0.5775 - 53s/epoch - 110ms/step
Epoch 7/10
481/481 - 53s - loss: 0.6702 - accuracy: 0.6092 - 53s/epoch - 110ms/step
Epoch 8/10
481/481 - 53s - loss: 0.6669 - accuracy: 0.6386 - 53s/epoch - 110ms/step
Epoch 9/10
481/481 - 53s - loss: 0.6635 - accuracy: 0.6539 - 53s/epoch - 111ms/step
Epoch 10/10
481/481 - 53s - loss: 0.6600 - accuracy: 0.6744 - 53s/epoch - 111ms/step
accuracy: 67.55%


Model Accuracy List:  [69.33766007423401, 70.37776708602905, 68.27827095985413, 64.808189868927, 67.55430102348328]
68.07% (+/- 1.89%)


In [ ]:
#LSTM 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):                                     
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_test[test_index], y_test[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_fold_after_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_after.sav"
pickle.dump(model, open(filename, 'wb'))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 58s - loss: 0.3683 - accuracy: 0.8278 - 58s/epoch - 121ms/step
Epoch 2/10
481/481 - 53s - loss: 0.1449 - accuracy: 0.9459 - 53s/epoch - 111ms/step
Epoch 3/10
481/481 - 53s - loss: 0.1234 - accuracy: 0.9550 - 53s/epoch - 111ms/step
Epoch 4/10
481/481 - 53s - loss: 0.0788 - accuracy: 0.9688 - 53s/epoch - 110ms/step
Epoch 5/10
481/481 - 53s - loss: 0.0683 - accuracy: 0.9727 - 53s/epoch - 111ms/step
Epoch 6/10
481/481 - 53s - loss: 0.0539 - accuracy: 0.9798 - 53s/epoch - 110ms/step
Epoch 7/10
481/481 - 53s - loss: 0.0529 - accuracy: 0.9808 - 53s/epoch - 110ms/step
Epoch 8/10
481/481 - 53s - loss: 0.0415 - accuracy: 0.9842 - 53s/epoch - 110ms/step
Epoch 9/10
481/481 - 53s - loss: 0.0358 - accuracy: 0.9859 - 53s/epoch - 110ms/step
Epoch 10/10
481/481 - 53s - loss: 0.0322 - accuracy: 0.9875 - 53s/epoch - 110ms/step
accuracy: 99.03%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

481/481 - 57s - loss: 0.3744 - accuracy: 0.8224 - 57s/epoch - 118ms/step
Epoch 2/10
481/481 - 52s - loss: 0.1429 - accuracy: 0.9469 - 52s/epoch - 109ms/step
Epoch 3/10
481/481 - 54s - loss: 0.1131 - accuracy: 0.9565 - 54s/epoch - 112ms/step
Epoch 4/10
481/481 - 53s - loss: 0.0796 - accuracy: 0.9671 - 53s/epoch - 109ms/step
Epoch 5/10
481/481 - 52s - loss: 0.0855 - accuracy: 0.9697 - 52s/epoch - 109ms/step
Epoch 6/10
481/481 - 52s - loss: 0.0529 - accuracy: 0.9797 - 52s/epoch - 109ms/step
Epoch 7/10
481/481 - 53s - loss: 0.0487 - accuracy: 0.9818 - 53s/epoch - 109ms/step
Epoch 8/10
481/481 - 53s - loss: 0.0381 - accuracy: 0.9854 - 53s/epoch - 109ms/step
Epoch 9/10
481/481 - 52s - loss: 0.0342 - accuracy: 0.9869 - 52s/epoch - 109ms/step
Epoch 10/10
481/481 - 53s - loss: 0.0456 - accuracy: 0.9835 - 53s/epoch - 110ms/step
accuracy: 99.05%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


481/481 - 56s - loss: 0.3631 - accuracy: 0.8278 - 56s/epoch - 117ms/step
Epoch 2/10
481/481 - 53s - loss: 0.1454 - accuracy: 0.9450 - 53s/epoch - 110ms/step
Epoch 3/10
481/481 - 53s - loss: 0.1124 - accuracy: 0.9574 - 53s/epoch - 110ms/step
Epoch 4/10
481/481 - 53s - loss: 0.0845 - accuracy: 0.9661 - 53s/epoch - 110ms/step
Epoch 5/10
481/481 - 53s - loss: 0.0688 - accuracy: 0.9719 - 53s/epoch - 110ms/step
Epoch 6/10
481/481 - 53s - loss: 0.0562 - accuracy: 0.9788 - 53s/epoch - 109ms/step
Epoch 7/10
481/481 - 53s - loss: 0.0497 - accuracy: 0.9811 - 53s/epoch - 109ms/step
Epoch 8/10
481/481 - 53s - loss: 0.0539 - accuracy: 0.9807 - 53s/epoch - 110ms/step
Epoch 9/10
481/481 - 53s - loss: 0.0368 - accuracy: 0.9859 - 53s/epoch - 110ms/step
Epoch 10/10
481/481 - 53s - loss: 0.0364 - accuracy: 0.9859 - 53s/epoch - 110ms/step
accuracy: 99.28%


Model Accuracy List:  [99.02645945549011, 98.71026873588562, 99.26770329475403, 99.05134439468384, 99.27602410316467]
99.07% (+/- 0.21%)


In [ ]:
#RNN FOLD 30% BEFORE TUNING======================
file_rnn_fold_30_data_before = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_fold_before_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_30_data_before = [(pickle.load(open(file_rnn_fold_30_data_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_30_data_before = [(loaded_model_rnn_30_data_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_30_data_before = [(np.argmax(y_pred_rnn_30_data_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_30_data_before[ypvc] = np.reshape(y_pred_values_rnn_30_data_before[ypvc],(-1,1))
result_rnn_30_data_before = [(classification_report(y_test, y_pred_values_rnn_30_data_before[res], output_dict = True)) for res in range(total_fold)]

#RNN FOLD 30% AFTER TUNING=======================
file_rnn_fold_30_data_after = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_30%data_fold_after_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_30_data_after = [(pickle.load(open(file_rnn_fold_30_data_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_30_data_after = [(loaded_model_rnn_30_data_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_30_data_after = [(np.argmax(y_pred_rnn_30_data_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_30_data_after[ypvc] = np.reshape(y_pred_values_rnn_30_data_after[ypvc],(-1,1))
result_rnn_30_data_after = [(classification_report(y_test, y_pred_values_rnn_30_data_after[res], output_dict = True)) for res in range(total_fold)]

#GRU 30% FOLD BEFORE TUNING======================
file_gru_fold_30_data_before = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_fold_before_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_30_data_before = [(pickle.load(open(file_gru_fold_30_data_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_30_data_before = [(loaded_model_gru_30_data_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_30_data_before = [(np.argmax(y_pred_gru_30_data_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_30_data_before[ypvc] = np.reshape(y_pred_values_gru_30_data_before[ypvc],(-1,1))
result_gru_30_data_before = [(classification_report(y_test, y_pred_values_gru_30_data_before[res], output_dict = True)) for res in range(total_fold)]

#GRU 30% FOLD AFTER TUNING=======================
file_gru_fold_30_data_after = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_30%data_fold_after_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_30_data_after = [(pickle.load(open(file_gru_fold_30_data_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_30_data_after = [(loaded_model_gru_30_data_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_30_data_after = [(np.argmax(y_pred_gru_30_data_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_30_data_after[ypvc] = np.reshape(y_pred_values_gru_30_data_after[ypvc],(-1,1))
result_gru_30_data_after = [(classification_report(y_test, y_pred_values_gru_30_data_after[res], output_dict = True)) for res in range(total_fold)]

#LSTM 30% FOLD BEFORE TUNING========================= 
file_lstm_fold_30_data_before = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_fold_before_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_30_data_before = [(pickle.load(open(file_lstm_fold_30_data_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_30_data_before = [(loaded_model_lstm_30_data_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_30_data_before = [(np.argmax(y_pred_lstm_30_data_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_30_data_before[ypvc] = np.reshape(y_pred_values_lstm_30_data_before[ypvc],(-1,1))
result_lstm_30_data_before = [(classification_report(y_test, y_pred_values_lstm_30_data_before[res], output_dict = True)) for res in range(total_fold)]

#LSTM 30% FOLD AFTER TUNING========================= 
file_lstm_fold_30_data_after = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_30%data_fold_after_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_30_data_after = [(pickle.load(open(file_lstm_fold_30_data_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_30_data_after = [(loaded_model_lstm_30_data_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_30_data_after = [(np.argmax(y_pred_lstm_30_data_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_30_data_after[ypvc] = np.reshape(y_pred_values_lstm_30_data_after[ypvc],(-1,1))
result_lstm_30_data_after = [(classification_report(y_test, y_pred_values_lstm_30_data_after[res], output_dict = True)) for res in range(total_fold)]

1878/1878 [==============================] - 38s 20ms/step


In [ ]:
precision_list_30_data_before = {}
precision_list_30_data_after = {}
recall_list_30_data_before = {}
recall_list_30_data_after = {}
accuracy_list_30_data_before = {}
accuracy_list_30_data_after = {}
f1_list_30_data_before = {}
f1_list_30_data_after = {}

In [ ]:
for sco in range(total_fold):
  precision_list_30_data_before[(sco+1)] = [result_rnn_30_data_before[sco]['weighted avg']['precision'] , result_gru_30_data_before[sco]['weighted avg']['precision'], result_lstm_30_data_before[sco]['weighted avg']['precision']]
  precision_list_30_data_after[(sco+1)] = [result_rnn_30_data_after[sco]['weighted avg']['precision'] , result_gru_30_data_after[sco]['weighted avg']['precision'], result_lstm_30_data_after[sco]['weighted avg']['precision']]
  recall_list_30_data_before[(sco+1)] = [result_rnn_30_data_before[sco]['weighted avg']['recall'] , result_gru_30_data_before[sco]['weighted avg']['recall'], result_lstm_30_data_before[sco]['weighted avg']['recall']]
  recall_list_30_data_after[(sco+1)] = [result_rnn_30_data_after[sco]['weighted avg']['recall'] , result_gru_30_data_after[sco]['weighted avg']['recall'], result_lstm_30_data_after[sco]['weighted avg']['recall']]
  f1_list_30_data_before[(sco+1)] = [result_rnn_30_data_before[sco]['weighted avg']['f1-score'] , result_gru_30_data_before[sco]['weighted avg']['f1-score'], result_lstm_30_data_before[sco]['weighted avg']['f1-score']]
  f1_list_30_data_after[(sco+1)] = [result_rnn_30_data_after[sco]['weighted avg']['f1-score'] , result_gru_30_data_after[sco]['weighted avg']['f1-score'], result_lstm_30_data_after[sco]['weighted avg']['f1-score']]
  accuracy_list_30_data_before[(sco+1)] = [result_rnn_30_data_before[sco]['accuracy'] , result_gru_30_data_before[sco]['accuracy'], result_lstm_30_data_before[sco]['accuracy']]
  accuracy_list_30_data_after[(sco+1)] = [result_rnn_30_data_after[sco]['accuracy'] , result_gru_30_data_after[sco]['accuracy'], result_lstm_30_data_after[sco]['accuracy']]

In [ ]:
print("{:<5} {:<0}".format("","Precision K-Fold 30% Data Sebelum Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_30_data_before.items():
    rnn_fold_data_before, gru_fold_data_before, lstm_fold_data_before = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_before,".3f"), format(gru_fold_data_before,".3f"), format(lstm_fold_data_before,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold 30% Data Sesudah Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_30_data_after.items():
    rnn_fold_data_after, gru_fold_data_after, lstm_fold_data_after = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_before,".3f"), format(gru_fold_data_before,".3f"), format(lstm_fold_data_before,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Recall K-Fold 30% Data Sebelum Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_30_data_before.items():
    rnn_fold_data_before, gru_fold_data_before, lstm_fold_data_before = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_before,".3f"), format(gru_fold_data_before,".3f"), format(lstm_fold_data_before,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold 30% Data Sesudah Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_30_data_after.items():
    rnn_fold_data_after, gru_fold_data_after, lstm_fold_data_after = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_after,".3f"), format(gru_fold_data_after,".3f"), format(lstm_fold_data_after,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Accuracy K-Fold 30% Data Sebelum Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_30_data_before.items():
    rnn_fold_data_before, gru_fold_data_before, lstm_fold_data_before = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_before,".3f"), format(gru_fold_data_before,".3f"), format(lstm_fold_data_before,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold 30% Data Sesudah Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_30_data_after.items():
    rnn_fold_data_after, gru_fold_data_after, lstm_fold_data_after = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_after,".3f"), format(gru_fold_data_after,".3f"), format(lstm_fold_data_after,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","F1-Score K-Fold 30% Data Sebelum Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_30_data_before.items():
    rnn_fold_data_before, gru_fold_data_before, lstm_fold_data_before = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_before,".3f"), format(gru_fold_data_before,".3f"), format(lstm_fold_data_before,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","F1-Score K-Fold 30% Data Sesudah Tuning"))
print ("{:<8} {:<10} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_30_data_after.items():
    rnn_fold_data_after, gru_fold_data_after, lstm_fold_data_after = v
    print ("{:<8} {:<10} {:<10} {:<10}".format(k, format(rnn_fold_data_after,".3f"), format(gru_fold_data_after,".3f"), format(lstm_fold_data_after,".3f") ))
print("")
print("")

      Precision K-Fold 30% Data Sebelum Tuning
Fold     RNN        GRU        LSTM      
1        0.900      0.597      0.727     
2        0.865      0.586      0.744     
3        0.845      0.620      0.693     
4        0.870      0.596      0.661     
5        0.738      0.641      0.679     


      Precision K-Fold 30% Data Sesudah Tuning
Fold     RNN        GRU        LSTM      
1        0.738      0.641      0.679     
2        0.738      0.641      0.679     
3        0.738      0.641      0.679     
4        0.738      0.641      0.679     
5        0.738      0.641      0.679     


      Recall K-Fold 30% Data Sebelum Tuning
Fold     RNN        GRU        LSTM      
1        0.897      0.594      0.690     
2        0.865      0.580      0.702     
3        0.845      0.602      0.680     
4        0.845      0.593      0.653     
5        0.723      0.616      0.673     


      Recall K-Fold 30% Data Sesudah Tuning
Fold     RNN        GRU        LSTM      
1        0.970